In [1]:
!pip install selenium 

  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 5.8 MB/s eta 0:00:00a 0:00:01
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 

In [4]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re

driver_path = "/usr/local/bin/chromedriver"
# 1. 크롬 드라이버 설정
service = Service(driver_path)  # 자신의 환경에 맞게 경로 수정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

# 2. 아마존 상품 URL
driver.get("https://www.amazon.com/dp/B06Y15D1LH")
time.sleep(3)


In [5]:
# 3. 리뷰 탭 클릭
driver.find_element(By.ID, "acrCustomerReviewLink").click()
time.sleep(3)

In [6]:


# 4. 리뷰 수집
reviews = []
page = 1
max_pages = 5  # 최대 5페이지까지 수집

while page <= max_pages:
    print(f"📄 페이지 {page} 수집 중...")

    review_elements = driver.find_elements(By.CSS_SELECTOR, "li[data-hook='review']")
    for r in review_elements:
        try:
            # 별점
            try:
                rating_text = r.find_element(By.CSS_SELECTOR, "[data-hook='review-star-rating']").text
            except:
                rating_text = r.find_element(By.CSS_SELECTOR, "[data-hook='review-rating']").text
            rating = float(re.search(r"(\d+(\.\d+)?)", rating_text).group(1)) if rating_text else None

            # 제목
            title = r.find_element(By.CSS_SELECTOR, "[data-hook='review-title']").text.strip()

            # 날짜
            date = r.find_element(By.CSS_SELECTOR, "[data-hook='review-date']").text.strip()

            # 본문
            body = r.find_element(By.CSS_SELECTOR, "[data-hook='review-collapsed']").text.strip()

            reviews.append({
                "rating": rating,
                "title": title,
                "date": date,
                "body": body
            })

        except Exception as e:
            print("❌ 리뷰 파싱 실패:", e)
            continue

    # 다음 페이지 버튼 클릭
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
        next_button.click()
        page += 1
        time.sleep(3)
    except NoSuchElementException:
        print("⛔️ 더 이상 다음 페이지가 없습니다.")
        break

# 5. 결과 보기
df = pd.DataFrame(reviews)
print(df)

# 6. 저장
df.to_csv("amazon_reviews.csv", index=False)

driver.quit()


📄 페이지 1 수집 중...
❌ 리뷰 파싱 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-hook='review-rating']"}
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104f12728 chromedriver + 5891880
1   chromedriver                        0x0000000104f09e7a chromedriver + 5856890
2   chromedriver                        0x00000001049da400 chromedriver + 418816
3   chromedriver                        0x0000000104a2c22b chromedriver + 754219
4   chromedriver                        0x0000000104a2c441 chromedriver + 754753
5   chromedriver                        0x0000000104a1fdd6 chromedriver + 703958
6   chromedriver                        0x0000000104a5250d chromedriver + 910605
7   chromedriver                        0x0000000104a1fcc8 chromedriver + 703688
8   ch

In [7]:
df

,rating,title,date,body
0,None,The most amazing hydrating product - not just ...,"Reviewed in the United States on May 25, 2025",I have very dry skin and since entering my 40'...
1,None,Love this product!!! It hydrates my skin like ...,"Reviewed in the United States on May 26, 2025",I purchased this as a last ditch effort to fin...
2,None,Love it! Glad I received an authentic product....,"Reviewed in the United States on April 27, 2025",Tired of facecare products for sensitive skin ...
3,None,Good for oily and dry skin,"Reviewed in the United States on June 5, 2025",I have been using this for 2 months now and fe...
4,None,Buen tamaño,"Reviewed in the United States on May 29, 2025","No siento que disminuya las rojeces, que es la..."
5,None,My Staple,"Reviewed in the United States on May 21, 2025",I will be buying this product as long as it is...
6,None,Skin changing,"Reviewed in the United States on May 17, 2025",Absolutely incredible. I had gotten a sample i...
7,None,Loved this,"Reviewed in the United States on May 13, 2025",Used this while I was pregnant on my stomach a...
